(melody-extraction)=
# Pitch extraction

As seen in the melodic introduction of [Carnatic](carnatic-melodic-concepts) and [Hindustani](hindustani-raga), pitch time-series is a very relevant feature to tackle the melodic analysis of these music traditions, showing utility for many musicologically-relevant problems. Given the shortage of recordings for isolated predominant melodic instruments, this task has had to be tackled directly from mixture recordings. In this context, the task is referred as *predominant* pitch extraction (otherwise commonly referred as *vocal* pitch extraction when the source is the singing voice).

Historically, in the literature of computational analysis of Indian Art Music, this task has been tackled using knowledge-based approaches {cite}`rao_pitch_2010, salamon_pitch_2012`. Melodia {cite}`salamon_pitch_2012`, typically combined with additional post-processing steps {cite}`gulati_patterns_2016`, has been the preferred approach until today.

In [ ]:
## Installing (if not) and importing compiam to the project
import importlib.util
if importlib.util.find_spec('compiam') is None:
    ## Bear in mind this will only run in a jupyter notebook / Collab session
    %pip install compiam
import compiam

# Import extras and supress warnings to keep the tutorial clean
import os
import numpy as np
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

Let's first print out the available tools we do have available to extract the pitch from Indian Art Music recordings.

In [ ]:
pprint(compiam.melody.pitch_extraction.list_tools())


## Melodia

Let's extract the pitch from an audio sample using Melodia {cite}`salamon_pitch_2012`. This is a salience-based method, meaning that it captures the most salient melodic segments in the time-frequency representation of the input signal, keeping and connecting the segments that are more likely to belong to the predominant melody, using heuristic rules. Make sure you check out the paper for further detail!

We first need to install `essentia`, which is the optional dependency required to load this tool.

In [ ]:
%pip install essentia

In [ ]:
# Importing and initializing a melodia instance
from compiam.melody.pitch_extraction import Melodia
melodia = Melodia()  

# Running extraction for an example track
melodia_pitch_track = melodia.extract(
    os.path.join(
        "..", "audio", "mir_datasets", "saraga1.5_carnatic",
        "Live at Vani Mahal by Sanjay Subrahmanyan", "Sri Raghuvara Sugunaalaya",
        "Sanjay Subrahmanyan - Sri Raghuvara Sugunaalaya.mp3"
    )
)

In [ ]:
print("Shape of the output pitch:", np.shape(melodia_pitch_track))

We can infer from the shape the number of estimated pitch values, while the dimension 2 refers to ``(frequency values, time-stamps)``. Let us show the first 5 time-stamps here.

In [ ]:
pprint(list(melodia_pitch_track[:5, 0]))

Let's now print out the final 5 pitch values.

In [ ]:
pprint(list(melodia_pitch_track[-5:, 1]))

Melodia has been found, in the original paper experiments and also in the [MIREX campaign](https://nema.lis.illinois.edu/nema_out/mirex2011/results/ame/indian08/sg1results.html), to decently work on Indian Art Music samples. However, recent DL-based models have claimed the state-of-the-art for the task of pitch extraction. 


## FTANet-Carnatic

**Maybe we can use a Carnatic-trained version of one of these models to extract the pitch?** Let's now import a DL model that learns to automatically extract the predominant melody from audio recordings. We will use FTANet {cite}`yu_pitch_2021`. This is an attention-based network that laverages and fuses information from frequency and periodicity domains to capture the right pitch values for the predominant source. It learns to focus on a particular source, using an additional branch that helps reducing the false alarm rate (detecting pitch values that do not correspond to the source we target).

To train this model we need a dataset that includes mixture recordings plus annotated pitch time-series for the source we aim at capturing the pitch from. This FTANet instance is trained with the [Saraga Carnatic Melody Synth (SMCS)](https://zenodo.org/record/5553925).

In the documentation we observe that this model is based on `tensorflow`, therefore we must install this dependency before importing it.

In [ ]:
%pip install tensorflow==2.9.3

In [ ]:
from compiam.melody.pitch_extraction import FTANetCarnatic

```{admonition} This is a non-trained instance!
Directly importing a ML/DL-based tool from the corresponding task initialises a non-trained instance. However, if a ``*`` appears at the end of the tool name when running ``.list_tools()``, the pre-trained weights can be loaded using the ``compiam.load_model()`` wrapper.
```

In ths case, we are only interested in inference. Therefore, we might be able to load FTANet as an already trained model. For that, let's print the models with available weights to load in `compiam`.

In [ ]:
pprint(compiam.list_models())

**Cool! A Carnatic-trained FTANet is there.** Therefore, let's load it and run inference on an example track.

In [ ]:
# Disabling tensorflow warnings and debugging info
import os 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" 

# Importing tensorflow and disabling GPU usage
import tensorflow as tf
tf.config.set_visible_devices([], "GPU")

Let's first deactivate the GPU usage, since we assume no CUDA-capable GPU is available in most of the cases. We import `tensorflow` and set the visible GPU devices to none. We also disable the `tensorflow` warnings in order to keep the tutorial clean.

```{note}
If you have an available GPU to allocate the model, get the index of the GPU (probably 0 if you have only a single instance) and change ``tf.config.set_visible_devices([], "GPU")`` for ``os.environ["CUDA_VISIBLE_DEVICES"] = "0"``
```

In [ ]:
# Initializing an FTANet instance
ftanet_carnatic = compiam.load_model("melody:ftanet-carnatic")

# Predict!
ftanet_pitch_track = ftanet_carnatic.predict(
    os.path.join(
        "..", "audio", "mir_datasets", "saraga1.5_carnatic",
        "Live at Vani Mahal by Sanjay Subrahmanyan", "Sri Raghuvara Sugunaalaya",
        "Sanjay Subrahmanyan - Sri Raghuvara Sugunaalaya.mp3"
    ),
)

Let's visualise the extracted pitch tracks on top of the spectrogram of the input signal.

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

y, sr = librosa.load(
    os.path.join(
        "..", "audio", "mir_datasets", "saraga1.5_carnatic",
        "Live at Vani Mahal by Sanjay Subrahmanyan", "Sri Raghuvara Sugunaalaya",
        "Sanjay Subrahmanyan - Sri Raghuvara Sugunaalaya.mp3"
    )
)
y = y[:3*sr]  # Getting just the first 3 seconds for better visualizatiom

fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(15, 12))
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
img = librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=sr, ax=ax);
ax.set_ylim(0, 2000)
plt.plot(
    melodia_pitch_track[:, 0], melodia_pitch_track[:, 1],
    color="white", label="Melodia",
)
plt.plot(
    ftanet_pitch_track[:, 0], ftanet_pitch_track[:, 1],
    color="black",label="FTANet-Carnatic",
)
plt.legend()
plt.show()


## Some final thoughts

Again, going through the melodic introduction of [Carnatic](carnatic-melodic-concepts) and Hindustani Music, we note the importance of a reliable pitch extraction for the computational analysis of Indian Art Music. The task of pitch extraction has received quite a lot of attention recently and the state-of-the-art has been continuously moved forward, especially given the use of DL techniques. However, to the best of our knowledge, no DL-based pitch extraction has been trained or evaluated on Indian Art Music signals, the lack of data being the principal cause. The FTA-Net trained for Carnatic Music that we present in this walkthrough and that we have included in `compiam` has been trained with the [Saraga Carnatic Melody Synth (SMCS)](https://zenodo.org/record/5553925), a recently released dataset that includes several hours of Carnatic Music recordings annotated with ground-truth vocal pitch data that have been artificially compiled. 